In [38]:
import pandas as pd

In [39]:
train = pd.read_csv("_train.csv").sample(frac=1)
test = pd.read_csv("_test.csv")

In [40]:
print(train.head())

                                                msno  is_churn  city    bd  \
29294   320mSbUUvMGa5hW9gIPOz2oy0HAlmatZhETMh9iiU5I=         1  12.0  29.0   
931898  zzl9h2LZs1EjLRXEUQKK1di+wj08ISUBoTRvwF4Avzg=         0   1.0   0.0   
196034  nEVeI7DhUVuX+K4o/kZQ5RLR8TZsDwRnla+eThGBF7E=         0  22.0  21.0   
311396  8h5DMcOrzSEvdzW/Z/FNvL2bTK0gCGuNhFJHljg55v8=         0   1.0   0.0   
532650  xutDDYs0BU01bc9P3DGgBlPh8XGMLVpKZZWZoMSkB2s=         0  22.0  29.0   

        gender  registered_via  registration_init_time  expiration_date  \
29294      1.0             4.0              20161220.0       20170618.0   
931898  -999.0             7.0              20160731.0       20170930.0   
196034     1.0             9.0              20150705.0       20170914.0   
311396  -999.0             7.0              20130819.0       20170922.0   
532650     1.0             9.0              20070424.0       20170930.0   

        registration_init_year  registration_init_month    ...      \
29294     

In [41]:
from catboost import CatBoostClassifier

In [42]:
from bayes_opt import BayesianOptimization
from sklearn.cross_validation import cross_val_score

# Load data set and target values
data, target = train.drop(['is_churn', 'msno'], axis=1), train[["is_churn"]]

In [43]:

def catboostcv(d, l2, rate, i):
    return cross_val_score(CatBoostClassifier(
        depth=int(d), 
        l2_leaf_reg=int(l2), 
        learning_rate=rate,
        iterations=int(i),
        loss_function='Logloss',
        eval_metric='Logloss',
        random_seed=1337
    ), data, target, "neg_log_loss", cv=5).mean()

catboostBO = BayesianOptimization(catboostcv, 
                             {'d': (1, 15), 
                              'l2': (0, 3), 
                              'rate': (0.01, 0.1), 
                              'i': (10, 1000)})
catboostBO.explore({
    'd': [3, 10], 
    'l2': [2, 3], 
    'rate': [0.03, 0.06], 
    'i': [100, 800]})

catboostBO.maximize()

print('Final Results')
print('Catboost: %f' % catboostBO.res['max']['max_val'])

Initialization
-----------------------------------------------------------------------------
 Step |   Time |      Value |         d |         i |        l2 |      rate | 
    1 | 02m03s |   -0.11759 |    3.0000 |  100.0000 |    2.0000 |    0.0300 | 
    2 | 28m49s |   -0.09575 |   10.0000 |  800.0000 |    3.0000 |    0.0600 | 
    3 | 12m56s |   -0.09775 |    6.5756 |  532.6187 |    1.6307 |    0.0856 | 
    4 | 09m05s |   -0.09738 |    8.6526 |  306.8848 |    1.4063 |    0.0849 | 
    5 | 06m56s |   -0.10266 |    3.9950 |  393.6948 |    2.9044 |    0.0890 | 
    6 | 89m18s |   -0.09593 |   14.1343 |  912.6022 |    1.7099 |    0.0936 | 
    7 | 12m31s |   -0.09786 |    6.1350 |  517.2616 |    2.9461 |    0.0787 | 


/home/joseph/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
-----------------------------------------------------------------------------
 Step |   Time |      Value |         d |         i |        l2 |      rate | 
    8 | 56m38s |   -0.09849 |   14.9131 |  466.3308 |    2.8088 |    0.0111 | 


/home/joseph/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00018132]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 48, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/joseph/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -2.62621361e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


    9 | 13m14s |   -0.11181 |    1.1267 |  999.9575 |    2.2705 |    0.0983 | 
   10 | 74m24s |   -0.09562 |   14.9295 |  646.5290 |    2.9435 |    0.0850 | 
   11 | 26m03s |   -0.09621 |   14.9194 |  211.5645 |    2.8679 |    0.0750 | 
   12 | 11m34s |   -0.11884 |    1.0426 |  871.2562 |    2.9705 |    0.0298 | 


KeyboardInterrupt: 

In [44]:
model = CatBoostClassifier(
        depth=15, 
        l2_leaf_reg=3, 
        learning_rate=0.085,
        iterations=650,
        loss_function='Logloss',
        eval_metric='Logloss',
        random_seed=1337)

In [45]:
model.fit(
        train.drop(['is_churn', 'msno'], axis=1), train[["is_churn"]],
        verbose = True)

Borders for float features generated
0:	learn 0.522978062	total: 3.45s	remaining: 37m 20s
1:	learn 0.4038966852	total: 6.93s	remaining: 37m 26s
2:	learn 0.3218978533	total: 8.11s	remaining: 29m 8s
3:	learn 0.260479897	total: 11.5s	remaining: 30m 49s
4:	learn 0.2223256358	total: 12.2s	remaining: 26m 9s
5:	learn 0.1919585515	total: 15.5s	remaining: 27m 43s
6:	learn 0.1699994919	total: 18.9s	remaining: 28m 52s
7:	learn 0.1547043767	total: 22.2s	remaining: 29m 41s
8:	learn 0.143633075	total: 25.5s	remaining: 30m 16s
9:	learn 0.1345014176	total: 28.8s	remaining: 30m 44s
10:	learn 0.1276458004	total: 32.1s	remaining: 31m 6s
11:	learn 0.1226649644	total: 34.9s	remaining: 30m 54s
12:	learn 0.1188568819	total: 38.2s	remaining: 31m 11s
13:	learn 0.1155115104	total: 41.5s	remaining: 31m 25s
14:	learn 0.1128007998	total: 44.8s	remaining: 31m 37s
15:	learn 0.1107767412	total: 46s	remaining: 30m 21s
16:	learn 0.1090987975	total: 49.2s	remaining: 30m 33s
17:	learn 0.1077275191	total: 52.6s	remaining:

145:	learn 0.09113333354	total: 7m 10s	remaining: 24m 46s
146:	learn 0.09107263373	total: 7m 13s	remaining: 24m 44s
147:	learn 0.09096465312	total: 7m 17s	remaining: 24m 43s
148:	learn 0.09084075159	total: 7m 20s	remaining: 24m 42s
149:	learn 0.09082376817	total: 7m 22s	remaining: 24m 34s
150:	learn 0.09077675804	total: 7m 25s	remaining: 24m 32s
151:	learn 0.09073156916	total: 7m 29s	remaining: 24m 31s
152:	learn 0.09067595047	total: 7m 32s	remaining: 24m 29s
153:	learn 0.09065432555	total: 7m 35s	remaining: 24m 27s
154:	learn 0.09054248135	total: 7m 39s	remaining: 24m 26s
155:	learn 0.09048768482	total: 7m 42s	remaining: 24m 24s
156:	learn 0.09042763048	total: 7m 45s	remaining: 24m 22s
157:	learn 0.0904168906	total: 7m 46s	remaining: 24m 12s
158:	learn 0.09038842162	total: 7m 49s	remaining: 24m 10s
159:	learn 0.09035264057	total: 7m 53s	remaining: 24m 8s
160:	learn 0.09033105575	total: 7m 56s	remaining: 24m 6s
161:	learn 0.09024918624	total: 7m 59s	remaining: 24m 4s
162:	learn 0.09018

286:	learn 0.08454983262	total: 14m 54s	remaining: 18m 51s
287:	learn 0.08448443517	total: 14m 57s	remaining: 18m 48s
288:	learn 0.08440658493	total: 15m 1s	remaining: 18m 45s
289:	learn 0.08436667922	total: 15m 4s	remaining: 18m 43s
290:	learn 0.08435016841	total: 15m 8s	remaining: 18m 40s
291:	learn 0.08432521563	total: 15m 11s	remaining: 18m 37s
292:	learn 0.08426237112	total: 15m 14s	remaining: 18m 34s
293:	learn 0.08419881593	total: 15m 18s	remaining: 18m 31s
294:	learn 0.08413416839	total: 15m 21s	remaining: 18m 28s
295:	learn 0.08409666283	total: 15m 24s	remaining: 18m 26s
296:	learn 0.08407709418	total: 15m 28s	remaining: 18m 23s
297:	learn 0.08406926346	total: 15m 31s	remaining: 18m 20s
298:	learn 0.08403575382	total: 15m 34s	remaining: 18m 17s
299:	learn 0.08402331752	total: 15m 38s	remaining: 18m 14s
300:	learn 0.08400391119	total: 15m 41s	remaining: 18m 11s
301:	learn 0.0839992063	total: 15m 44s	remaining: 18m 8s
302:	learn 0.08396831537	total: 15m 47s	remaining: 18m 5s
303

426:	learn 0.0807899251	total: 22m 18s	remaining: 11m 38s
427:	learn 0.08078981456	total: 22m 21s	remaining: 11m 35s
428:	learn 0.08078335783	total: 22m 24s	remaining: 11m 32s
429:	learn 0.08077723352	total: 22m 27s	remaining: 11m 29s
430:	learn 0.08077716695	total: 22m 29s	remaining: 11m 25s
431:	learn 0.08075810685	total: 22m 32s	remaining: 11m 22s
432:	learn 0.08074921116	total: 22m 36s	remaining: 11m 19s
433:	learn 0.0807287437	total: 22m 39s	remaining: 11m 16s
434:	learn 0.08072869181	total: 22m 42s	remaining: 11m 13s
435:	learn 0.08072859212	total: 22m 45s	remaining: 11m 10s
436:	learn 0.08071193588	total: 22m 48s	remaining: 11m 7s
437:	learn 0.0806894292	total: 22m 52s	remaining: 11m 4s
438:	learn 0.08067969523	total: 22m 55s	remaining: 11m 1s
439:	learn 0.08067740544	total: 22m 58s	remaining: 10m 57s
440:	learn 0.0806549995	total: 23m 1s	remaining: 10m 54s
441:	learn 0.08062264279	total: 23m 5s	remaining: 10m 51s
442:	learn 0.08061072388	total: 23m 8s	remaining: 10m 48s
443:	le

568:	learn 0.07939955661	total: 27m 50s	remaining: 3m 57s
569:	learn 0.07939952785	total: 27m 51s	remaining: 3m 54s
570:	learn 0.07936447825	total: 27m 54s	remaining: 3m 51s
571:	learn 0.07935337629	total: 27m 57s	remaining: 3m 48s
572:	learn 0.079350263	total: 28m 1s	remaining: 3m 45s
573:	learn 0.07935024276	total: 28m 2s	remaining: 3m 42s
574:	learn 0.07932964664	total: 28m 5s	remaining: 3m 39s
575:	learn 0.07930478884	total: 28m 8s	remaining: 3m 36s
576:	learn 0.07929133064	total: 28m 12s	remaining: 3m 34s
577:	learn 0.07928847353	total: 28m 15s	remaining: 3m 31s
578:	learn 0.07926908315	total: 28m 18s	remaining: 3m 28s
579:	learn 0.07926895409	total: 28m 21s	remaining: 3m 25s
580:	learn 0.07924516335	total: 28m 25s	remaining: 3m 22s
581:	learn 0.07923011854	total: 28m 28s	remaining: 3m 19s
582:	learn 0.07919800203	total: 28m 31s	remaining: 3m 16s
583:	learn 0.07919443055	total: 28m 35s	remaining: 3m 13s
584:	learn 0.07919439341	total: 28m 38s	remaining: 3m 10s
585:	learn 0.0791441

In [46]:
# print(test.head())
result = model.predict(test.drop(["msno", "is_churn"], axis = 1))

In [47]:
test[["is_churn"]] = result

In [48]:
(test[["msno", "is_churn"]]).to_csv("submission.csv", index = False)